# Building a Recommendation Engine from Scratch
Agenda today:
1. Different types of recommendation engines
2. __Building our own hotel recommendation system__
    - Package Installation
    - Package Import
    - Performing exploratory data analysis
    - Perform feature engineering
    - Make recommendation 
    - Compare our engines with others

In [ ]:
# installing packages
# !pip install numpy
# !pip install pandas
# !pip install surprise
# !pip install nltk

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
import random
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 


#from surprise import Dataset, Reader
#from surprise import SVD
#from surprise import accuracy
#from surprise.model_selection import cross_validate, train_test_split

## 1. Reading in the data and a simple exploratory data analysis

In [ ]:
data = pd.read_csv("hotels.csv", encoding="latin-1")

In [ ]:
# check out the shape and top few observations of the data
data.shape
data.head(2)

In [ ]:
# count the common length of description
data['desc_length'] = data.desc.apply(lambda x:len(x.split(" ")))

In [ ]:
data.desc_length.describe()

In [ ]:

print("There are {} documents in total".format(data.desc_length.describe()['count']))
print("The longest document has {} words".format(data.desc_length.describe()['max']))
print("The shortest document has {} words".format(data.desc_length.describe()['min']))

In [ ]:
# examining the distribution of the document lengths using histograms

This indicates that in our dataset, we have 152 hotels, and 3 features that describe our data: the names, the address, and the description.

In [ ]:
# let's check out whether there is any duplicates
len(data.name.unique())

In [ ]:
# getting the word frequency of the description
word_freq = data.desc.str.split(expand=True).stack().value_counts()

In [ ]:
word_freq[:20]

In [ ]:
# create a bar graph of the word frequency 
word_freq_top_20 = dict(word_freq[:20])
plt.figure(figsize = (14,14))
plt.bar(range(len(word_freq_top_20)), word_freq_top_20.values(), tick_label=list(word_freq_top_20.keys()))

### Question: are these information going to be helpful to us when we are making recommendation? Why or why not?
We need to do some serious __data cleaning__!

In [ ]:
# first of all, let us remove some useless words to see how our documents change
stop_words = set(stopwords.words('english')) 
stop_words

In [ ]:
# we will then remove stop words from the hotel description to clean up the data 
data = data[['name','address','desc','desc_length']]
data.head()

In [ ]:
# we will do some data cleaning
data['desc_lower'] = data.desc.apply(lambda x:x.lower())
data['without_stopwords'] = data['desc_lower'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
data.head()

In [ ]:
# comparing description with stopwords and description without stopwords
print(data.without_stopwords[0])
print('------------------------')
print(data.desc[0])

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    return text
    
data['desc_complete_cleaned'] = data['without_stopwords'].apply(clean_text)


In [ ]:
# compare the cleaned version of description
data['desc_complete_cleaned'][0]

In [ ]:
word_freq_clean = data.desc_complete_cleaned.str.split(expand=True).stack().value_counts()
word_freq_clean[:20]
plt.figure(figsize=(16,16))
plt.bar(range(len(word_freq_clean[:20])), dict(word_freq_clean[:20]).values(), tick_label=list(dict(word_freq_clean[:20]).keys()))


### Making recommendation

In [ ]:
data.set_index('name', inplace = True)

In [ ]:
# calculate cosine similarity between documents - we use tfidf
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(data['desc_complete_cleaned'])
similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
# examine the dense version of the documents 
#doc_term_matrix = tfidf_matrix.todense()
#df_term = pd.DataFrame(doc_term_matrix, 
                  #columns=tf.get_feature_names())
#df_term.shape

In [ ]:
print(similarity_matrix)

In [ ]:
indices = pd.Series(data.index)

In [ ]:
indices[indices == "Vermont Inn"].index[0]

In [ ]:
def recommendations(name, similarity_matrix = similarity_matrix):
    
    recommended_hotels = []
    
    # gettin the index of the hotel that matches the name
    idx = indices[indices == name].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(similarity_matrix[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar hotels except itself
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the names of the top 10 matching hotels
    for i in top_10_indexes:
        recommended_hotels.append(list(data.index)[i])
        
    return recommended_hotels

In [ ]:
# get the list of optional names 
data.index

In [ ]:
recomendations = input("what types of hotels would you like me to recommend for ya?")
recommendations(recomendations)
